In [ ]:
%load_ext autoreload
%autoreload 2

from gpu import set_gpu
import numpy as np
import os
import adapt.utils.data_utils as prd
import adapt.loop as lp
import adapt.ml.lda as dlda
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import adapt.ml.dl_subclass as dl
import copy as cp
import tensorflow as tf
import pickle
from datetime import datetime
import time
from IPython import display
import gc as gc

set_gpu()

In [5]:
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
bat = 32
load_mod = False
mod_tot = ['blda','lda','crlda','alda','bcnn','cnn', 'crcnn2','acnn03','acnn30','acewc30','acewc15', 'adcnn30', 'vcnn', 'bvcnn', 'avcnn', 'avcnn15', 'acnnl03','crvcnn','acewclm','crcnn','acewc00','xtra2']
ft = 'tdar'
iter = 1
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_mod = 0

for it in range(iter):
    # first iteration, includes LDA; others exclude LDA
    if it == 0:
        mod_all = ['lda','blda','alda','bcnn','cnn','acnn03','avcnn']#,'crcnn']
        # mod_all = ['avcnn']
    else:
        mod_all = ['bcnn','cnn','acnn03','avcnn','crcnn']

    for sub in range(3,7):
        print(subs[sub])
        sub_path = path + subs[sub] + '/DATA/MAT/'
        all_files = os.listdir(sub_path)
        if 'skip' in all_files:
            all_files = np.delete(all_files,all_files.index('skip'))

        # load or initialize cnn weights
        if load_mod:
            with open(subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
                all_acc, all_recal, all_val, all_prev, all_train, all_times, _, _, c_weights, v_weights, cl_wc, scaler_0, emg_scale = pickle.load(f)
        else:
            c_weights = None
            v_weights = None
            v_wc = None
            cl_wc = None
            scaler_0 = None
            all_recal = np.zeros((len(all_files),len(mod_tot)))
            all_acc = np.zeros((len(all_files),len(mod_tot)))
            all_val = np.zeros((len(all_files),len(mod_tot)))
            all_prev = np.zeros((len(all_files),len(mod_tot)))
            all_train = np.zeros((len(all_files),len(mod_tot)))
            all_times = np.zeros((len(all_files),len(mod_tot)))

        mod_i = 0
        for mod in mod_all:
            acc = np.zeros((len(all_files),2))
            acc_val = np.zeros((len(all_files),2))
            acc_prev = np.zeros((len(all_files),2))
            acc_train = np.zeros((len(all_files),2))
            mod_recal = np.zeros((len(all_files),))

            if 'lda' in mod:
                acc_i = 0
            else:
                acc_i = 1

            cnn = None
            ewc = None
            clda = None

            ep = 30
            recal = 0
            skip_recal = 0
            skip = False

            # Loop through files
            for i in range(1,len(all_files)-1):
                # Check if need to recalibrate
                if i > 1:
                    if acc[i,acc_i] < 75:
                        skip = False
                        recal += 1
                        print('recal: ' + str(recal) + ' ' + all_files[i])
                        acc[i,acc_i] *= -1
                        mod_recal[i] = 1
                    else:
                        skip = True
                    
                    if 'b' in mod:
                        skip = True
                        
                if not skip:
                    # load training file
                    train_file = all_files[i]
                    train_data, train_params = prd.load_caps_train(sub_path + train_file + '/traindata.mat')

                    # if first train, use two train files
                    if i == 1:
                        train_data2, train_params2 = prd.load_caps_train(sub_path + all_files[i-1] + '/traindata.mat')
                        train_data = np.vstack((train_data,train_data2))
                        train_params = np.vstack((train_params,train_params2))
                        del train_data2, train_params2

                        train_data, train_params, th = prd.threshold(train_data, train_params)
                        val_data = train_data
                        val_params = train_params

                        # tr_i = np.zeros((train_params.shape[0],))
                        # te_i = np.zeros((train_params.shape[0],))
                        # for cls in np.unique(train_params[:,-1]):
                        #     dof = np.array(np.where(train_params[:,-1] == cls))
                        #     tr_i[dof[0,:dof.shape[1]//2]] = 1
                        #     te_i[dof[0,dof.shape[1]//2:]] = 1

                        # train_temp = train_data[tr_i.astype(bool),...]
                        # params_temp = train_params[tr_i.astype(bool),...]
                        # val_data = train_data[te_i.astype(bool),...]
                        # val_params = train_params[te_i.astype(bool),...]

                        # train_data, train_params = train_temp, params_temp

                        # del train_temp, params_temp, tr_i, te_i
                    else:
                        train_data, train_params, _ = prd.threshold(train_data, train_params,th)
                    
                        tr_i = np.zeros((train_params.shape[0],))
                        te_i = np.zeros((train_params.shape[0],))
                        for cls in np.unique(train_params[:,-1]):
                            dof = np.array(np.where(train_params[:,-1] == cls))
                            tr_i[dof[0,:dof.shape[1]//2]] = 1
                            te_i[dof[0,dof.shape[1]//2:]] = 1

                        train_temp = train_data[tr_i.astype(bool),...]
                        params_temp = train_params[tr_i.astype(bool),...]
                        val_data = train_data[te_i.astype(bool),...]
                        val_params = train_params[te_i.astype(bool),...]

                        train_data, train_params = train_temp, params_temp

                        del train_temp, params_temp, tr_i, te_i
                        
                    # if combining, save current training data
                    # if 'cr' in mod:
                    #     # combine old and new training data
                    #     if i > 1:
                    #         train_data = np.vstack((train_data_0,train_data))
                    #         train_params = np.vstack((train_params_0,train_params))

                    #     train_data_0 = cp.deepcopy(train_data)
                    #     train_params_0 = cp.deepcopy(train_params)

                    if (i == 1 and mod[0] == 'a') or (i == 1 and mod[:2] == 'cr') or (mod[0] != 'a' and mod[:2] != 'cr'):
                        if i > 1:
                            prev_ndof = [n_dof, key, train_dof]
                        train_dof = np.unique(train_params[:,-1])
                        key = np.empty(train_dof.shape)
                        for key_i in range(len(train_dof)):
                            key[key_i] = cp.deepcopy(train_params[np.argmax(train_params[:,2] == train_dof[key_i]),0])
                        n_dof = int(np.max(key))
                        if mod[0] == 'a' and 'cnn' in mod:
                            test_key = True
                        else:
                            test_key = False
                    else:
                        test_key = True

                    train_data, train_params = lp.check_labels(train_data,train_params,train_dof,key,test_key)
                    val_data, val_params = lp.check_labels(val_data,val_params,train_dof,key,test_key)

                    for cl in train_dof:
                        train_params[train_params[:,-1] == cl,0] = key[train_dof==cl]
                        val_params[val_params[:,-1] == cl,0] = key[train_dof==cl]

                    print('train dof:' ,end=' ')
                    print(train_dof)

                    if (mod[0] == 'a' and i > 1) or ('cr' in mod and i > 1) or (mod == 'vcnn' and i > 1):
                        _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, _, _, _, _, _ = prd.prep_train_caps(train_data, train_params, emg_scale=emg_scale, scaler=scaler, num_classes=n_dof, prop_b=False, batch_size=bat, ft=ft, noise=True, split=False)
                    else:
                        _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, emg_scale, scaler, _, _, _ = prd.prep_train_caps(train_data, train_params, prop_b=False, batch_size=bat, ft=ft, noise=True, split=False,num_classes=n_dof)
                        if ((i == 1) and (c_weights is not None)) or ((i == 1) and (v_weights is not None)):
                            scaler = cp.deepcopy(scaler_0)

                    _, _, _, _, x_val_cnn, y_val, x_val_lda, y_val_lda, _, _, _, _, _ = prd.prep_train_caps(val_data, val_params, emg_scale=emg_scale,scaler=scaler, prop_b=False, batch_size=bat, ft=ft, num_classes=n_dof, noise=False, split=False)

                    if 'cr' in mod:
                        # combine old and new training data
                        if i > 1:
                            x_clean_cnn = np.vstack((clean_data_0,x_clean_cnn))
                            y_clean = np.vstack((clean_params_0,y_clean))
                            x_train_cnn = np.vstack((x_clean_cnn,x_train_cnn))
                            y_train = np.vstack((y_clean,y_train))

                    del train_data, train_params, val_data, val_params

                    if 'lda' not in mod:
                        cnnlda = 'l' in mod
                        if 'vcnn' in mod:
                            if i == 1:
                                # calculate normalization scale
                                adjust = []
                                # for cl_i in range(n_dof):
                                #     adjust.append(np.std(x_clean_cnn[np.argmax(y_clean,axis=1)==cl_i,...],axis=0))
                                old_y = cp.deepcopy(y_clean)

                                if v_weights is None:
                                    cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_clean_cnn,y_train=y_clean, mod=['vcnn'], n_dof=n_dof, ep=30, dec=True, trainable=True)
                                    # save current autoencoder weights
                                    b_enc_w = cp.deepcopy(cnn.enc.get_weights())
                                    dec_w = cp.deepcopy(cnn.dec.get_weights())
                                    v_weights = cp.deepcopy(cnn.dec.get_weights())
                                else:
                                    cnn = dl.VCNN(n_class = n_dof)
                                    cnn(x_train_cnn[:1,...])
                                    cnn.add_dec(x_train_cnn[:1,...])
                                    cnn(x_train_cnn[:2,...],np.ones((2,)),dec=True) 
                                    cnn.dec.set_weights(v_weights)

                                if c_weights is None:
                                    cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=[cnn], n_dof=n_dof, ep=30, dec=False, trainable=True)
                                    c_weights = [cnn.enc.get_weights(),cnn.clf.get_weights()]
                                else:
                                    print('setting CNN weights')
                                    cnn.enc.set_weights(c_weights[0])
                                    cnn.clf.set_weights(c_weights[1])

                                if scaler_0 is None:
                                    scaler_0 = cp.deepcopy(scaler)
                                else:
                                    scaler = cp.deepcopy(scaler_0)

                                mu_class, std_class, N = prd.update_mean(x_clean_cnn,y_clean)

                                # prev_train = cp.deepcopy(x_clean_cnn)
                                # prev_trainy = cp.deepcopy(y_clean)
                            else:
                                prev_w = cnn.get_weights()
                                prev_mu = [mu_class, std_class, N]
                                # generate old training data, same size as clean data

                                x_out = cp.deepcopy(x_clean_cnn)
                                for cl in range((y_clean).shape[1]):
                                    # y_ind = prev_trainy[:,cl]==1
                                    x_ind = y_clean[:,cl]==1
                                    # x_out[x_ind,...] = np.random.normal(np.mean(prev_train[y_ind,...],axis=0), np.std(prev_train[y_ind,...],axis=0),x_clean_cnn[x_ind,...].shape)
                                    x_out[x_ind,...] = np.random.normal(mu_class[cl], std_class[cl],x_clean_cnn[x_ind,...].shape)

                                x_train_aug = np.vstack((x_out,x_train_cnn))
                                y_train_aug = np.vstack((y_clean,y_train))

                                x_clean_aug = np.vstack((x_out,x_clean_cnn))
                                y_clean_aug = np.vstack((y_clean,y_clean))

                                old_y = cp.deepcopy(y_clean_aug)
                                mu_class, std_class, N = prd.update_mean(x_clean_cnn,y_clean,N,mu_class,std_class)

                                # set new normalization scales
                                # for cl_i in range(n_dof):
                                #     adjust[cl_i] = np.std(x_clean_aug[np.argmax(y_clean_aug,axis=1)==cl_i,...],axis=0)

                                if 'avcnn' in mod: # update whole CNN and lda weights
                                    # ep = int(mod[-2:])
                                    # save current classifier weights
                                    # enc_w = cp.deepcopy(cnn.enc.get_weights())
                                    # clf_w = cp.deepcopy(cnn.clf.get_weights())

                                    # # set autoencoder weights, finetune and save new autoencoder weights
                                    # cnn.enc.set_weights(b_enc_w)
                                    # cnn.dec.set_weights(dec_w)
                                    # cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_clean_aug,y_train=y_clean_aug, mod=['vcnn'], n_dof=n_dof, ep=10, dec=True, lr=0.00001, trainable=False)
                                    # b_enc_w = cp.deepcopy(cnn.enc.get_weights())
                                    # dec_w = cp.deepcopy(cnn.dec.get_weights())

                                    # # set old classifier weights and finetune
                                    # cnn.enc.set_weights(enc_w)
                                    # cnn.clf.set_weights(clf_w)
                                    cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_aug,y_train=y_train_aug, mod=[cnn], n_dof=n_dof, ep=5, dec=False, lr=0.00001, trainable=False)
                                else:
                                    # cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_clean_aug,y_train=y_clean_aug, mod=['vcnn'], n_dof=n_dof, ep=30, dec=True, trainable=True)
                                    cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_aug,y_train=y_train_aug, mod=[cnn], n_dof=n_dof, ep=30, dec=False, trainable=True)

                                    w, c, _, _, _, _, _ = dlda.train_lda(x_train_aug.reshape((x_train_aug.shape[0],-1)), np.argmax(y_train_aug,axis=1)[...,np.newaxis])
                                
                                # prev_train = cp.deepcopy(x_clean_aug)
                                # prev_trainy = cp.deepcopy(y_clean_aug)
                            
                        else:
                            if i == 1:
                                if c_weights is None:
                                    cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['cnn'], n_dof=n_dof, ep=ep, cnnlda=cnnlda)
                                    c_weights = cp.deepcopy([cnn.enc.get_weights(),cnn.clf.get_weights()])
                                    scaler_0 = cp.deepcopy(scaler)    
                                else:
                                    print('setting CNN weights')
                                    cnn = dl.CNN(n_class=n_dof)
                                    cnn(x_train_cnn[:1,...])
                                    cnn.enc.set_weights(c_weights[0])
                                    cnn.clf.set_weights(c_weights[1])
                                    if cnnlda:
                                        print('setting LDA weights')
                                        w_c = cp.deepcopy(cl_wc[0].astype('float32'))
                                        c_c = cp.deepcopy(cl_wc[1].astype('float32'))
                                if 'ewc' in mod:
                                    cnn = dl.EWC(n_class=n_dof)
                                    cnn(x_train_cnn[:1,...])
                                    cnn.enc.set_weights(c_weights[0])
                                    cnn.clf.set_weights(c_weights[1])
                                if 'ad' in mod:
                                    cnn = dl.CNN(n_class=n_dof,adapt=True)
                                    cnn(x_train_cnn[:1,...])
                                    cnn.enc.set_weights(c_weights[0])
                                    cnn.clf.set_weights(c_weights[1])

                                if 'l' in mod:
                                    clda = [w_c, c_c]
                                
                                if scaler_0 is None:
                                    scaler_0 = cp.deepcopy(scaler)
                                else:
                                    scaler = cp.deepcopy(scaler_0)

                            else:
                                prev_w = cnn.get_weights()
                                if mod =='acnnlm': # update CNN encoder using lda for loss
                                    ep = 5
                                    cnn, all_times[i,mod_tot.index(mod)], _, _ = lp.train_models(traincnn=x_train_cnn,y_train=y_train, n_dof=n_dof, ep=ep, mod=[[cnn,w_c,c_c]],cnnlda=cnnlda)
                                elif 'adcnn' in mod: # adapt first layer only
                                    # cnn.base.trainable=False
                                    cnn.clf.trainable=False
                                    ep = int(mod[-2:])
                                    cnn, all_times[i,mod_tot.index(mod)], _, _ = lp.train_models(traincnn=x_train_cnn,y_train=y_train, n_dof=n_dof, ep=ep, mod=[cnn], adapt=True, cnnlda=cnnlda, lr=0.00001)
                                elif 'acnn' in mod: # update whole CNN and lda weights
                                    ep = int(mod[-2:])
                                    ep =5
                                    cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, n_dof=n_dof, ep=ep, mod=[cnn], cnnlda=cnnlda, lr=0.00001)
                                elif mod == 'afcnnl': # update lda only 
                                    w_c, c_c = lp.train_models(x_train_lda=cnn.enc(x_train_cnn).numpy(), y_train_lda=np.argmax(y_train,axis=1)[...,np.newaxis], mod=['lda'])
                                elif 'cnn' in mod: # recalibrate cnnlda
                                    cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['cnn'], n_dof=n_dof, ep=ep, cnnlda=cnnlda)
                                elif mod == 'acewclm':
                                    _, _, all_times[i,mod_tot.index(mod)] = lp.train_task(cnn, ep, 1, x_train_cnn, y_train, [x_val_cnn], [y_val], lams=[int(mod[-2:])], bat=bat, clda=[w_c,c_c], cnnlda=cnnlda)
                                elif 'acewc' in mod:
                                    w_c, c_c, all_times[i,mod_tot.index(mod)] = lp.train_task(cnn, 15, 1, x_train_cnn, y_train, [prev_x, x_val_cnn],[prev_y, y_val], lams=[int(mod[-2:])], bat=bat, cnnlda=cnnlda)
                                
                                if 'l' in mod:
                                    clda = [w_c, c_c]
                        del test_mod
                        test_mod = dl.get_test(cnn, test_accuracy)
                        if i > 1:
                            acc_prev[i,:] = lp.test_models(prev_x, prev_y, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)
                        acc_val[i,:] = lp.test_models(x_val_cnn, y_val, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)
                        acc_train[i,:] = lp.test_models(x_clean_cnn, y_clean, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)
                        
                        if acc_val[i,acc_i] < 75:
                            n_dof, key, train_dof = prev_ndof
                            if 'vcnn' in mod:
                                mu_class, std_class, N = prev_mu
                                cnn = dl.VCNN(n_class = n_dof)
                                cnn(x_train_cnn[:1,...])
                                cnn.add_dec(x_train_cnn[:1,...])
                                cnn(x_train_cnn[:2,...],np.ones((2,)),dec=True) 
                            elif 'cnn' in mod:
                                cnn = dl.CNN(n_class = n_dof)
                                cnn(x_train_cnn[:1,...])
                            cnn.set_weights(prev_w)
                            del test_mod
                            test_mod = dl.get_test(cnn, test_accuracy)
                            
                            mod_recal[i] = -1
                            print('bad recal')
                        elif 'cr' in mod:
                            clean_data_0 = cp.deepcopy(x_clean_cnn)
                            clean_params_0 = cp.deepcopy(y_clean)
                        if 'ewc' in mod: 
                            cnn.compute_fisher(x_train_cnn, y_train, num_samples=200, plot_diffs=False) 
                            cnn.star()
                    else:
                        print(N.shape)
                        if i == 1:
                            N = np.zeros((len(np.unique(y_train_lda)),))
                            cov_class = np.zeros([x_train_lda.shape[1],x_train_lda.shape[1]])
                            mu_class = np.zeros([len(np.unique(y_train_lda)),x_train_lda.shape[1]])
                        prev_lda = [mu_class,cov_class,N]
                        start_time = time.time()
                        if mod[0] != 'a' or (i == 1 and mod[0] == 'a'):
                            w, c, mu_class, _, _, N, cov_class = dlda.train_lda(x_train_lda, y_train_lda)
                        else:
                            w, c, mu_class, cov_class, N = dlda.update_lda(x_train_lda, y_train_lda, N, mu_class, cov_class)
                        all_times[i,mod_tot.index(mod)] = time.time() - start_time

                        acc_val[i,:] = lp.test_models(None, None, x_val_lda, y_val_lda, lda=[w,c])
                        acc_train[i,:] = lp.test_models(None, None, x_train_lda, y_train_lda, lda=[w,c])
                        if i > 1:
                            acc_prev[i,:] = lp.test_models(None, None, prev_x_lda, prev_y_lda, lda=[w,c])
                        if acc_val[i,acc_i] < 75:
                            mod_recal[i] = -1
                            mu_class, cov_class, N = prev_lda
                            n_dof, key, train_dof = prev_ndof
                            print('bad recal')
                        del x_train_lda, y_train_lda
                    
                    if mod_recal[i] != -1:
                        prev_x = cp.deepcopy(x_val_cnn)
                        prev_y = cp.deepcopy(y_val)
                        prev_x_lda = cp.deepcopy(x_val_lda)
                        prev_y_lda = cp.deepcopy(y_val_lda)
                    
                    del x_train_cnn, y_train, x_val_cnn, y_val, x_val_lda, y_val_lda, x_clean_cnn, y_clean
                
                # load data
                test_file = all_files[i+1]
                test_data, test_params = prd.load_caps_train(sub_path + test_file + '/traindata.mat')
                
                # check class labels
                test_data, test_params, _ = prd.threshold(test_data, test_params, th)
                test_data, test_params = lp.check_labels(test_data,test_params,train_dof,key)

                for cl in train_dof:
                    test_params[test_params[:,-1] == cl,0] = key[train_dof==cl]

                print('test dof:',end=' ')
                print(np.unique(test_params[:,-1]))
                # test 
                y_test, _, x_test_cnn, x_test_lda, y_test_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=n_dof, ft=ft, split=False)
                
                # test 
                if 'lda' in mod:
                    acc[i+1,:] = lp.test_models(None, None,  x_test_lda, y_test_lda, lda=[w,c])
                else:
                    acc[i+1,:] = lp.test_models(x_test_cnn, y_test, x_test_lda, y_test_lda, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)

                print ('Set: ' + train_file + ', Test: ' + test_file + ',', f'Accuracy: {acc[i+1,acc_i]:.2f}', f', Val: {acc_val[i,acc_i]:.2f}', f', Prev: {acc_prev[i,acc_i]:.2f}', f', Train: {acc_train[i,acc_i]:.2f}')
                del y_test, x_test_cnn, x_test_lda, y_test_lda, test_data, test_params

            all_acc[:,mod_tot.index(mod)] = acc[:,acc_i]
            # all_recal[mod_tot.index(mod),:] = np.hstack((recal,skip_recal))
            all_val[:,mod_tot.index(mod)] = acc_val[:,acc_i]
            all_prev[:,mod_tot.index(mod)] = acc_prev[:,acc_i]
            all_train[:,mod_tot.index(mod)] = acc_train[:,acc_i]
            all_recal[:,mod_tot.index(mod)] = mod_recal

            print(mod + ' ' + str(recal) + ' - ' + str(np.sum(mod_recal==-1)))
            mod_i += 1

            # if 'cr' in mod:
            #     del train_data_0, train_params_0

        with open(subs[sub] + '_' + str(it) + '_r_accs.p','wb') as f:
            pickle.dump([all_acc, all_recal, all_val, all_prev, all_train, all_times, mod_all, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale],f)
        
        gc.collect(generation=2)

TR59
train dof: [ 1  6 16 19 48]
(5,)
test dof: [ 1  6 16 19 48]
Set: 20170424_081251, Test: 20170424_082640, Accuracy: 40.44 , Val: 85.57 , Prev: 0.00 , Train: 85.57
recal: 1 20170424_082640
train dof: [ 1  6 16 19 48]
(5,)
bad recal
test dof: [ 1  6 16 19 48]
Set: 20170424_082640, Test: 20170424_085656, Accuracy: 65.69 , Val: 52.79 , Prev: 43.50 , Train: 95.64
recal: 2 20170424_085656
train dof: [ 1  6 16 19 48]
(5,)
test dof: [ 1  6 16 19 48]
Set: 20170424_085656, Test: 20170424_091501, Accuracy: 74.13 , Val: 89.40 , Prev: 52.93 , Train: 99.05
recal: 3 20170424_091501
train dof: [ 1  6 16 19 48]
(5,)
test dof: [ 1  6 16 19 48]
Set: 20170424_091501, Test: 20170424_094328, Accuracy: 69.84 , Val: 75.41 , Prev: 58.97 , Train: 99.32
recal: 4 20170424_094328
train dof: [ 1  6 16 19 48]
(5,)
bad recal
test dof: [ 1  6 16 19 48]
Set: 20170424_094328, Test: 20170424_104554, Accuracy: 27.84 , Val: 71.33 , Prev: 48.64 , Train: 98.09
recal: 5 20170424_104554
train dof: [ 1  6 16 19 48]
(5,)
bad

IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
bat = 32
load_mod = False
mod_tot = ['blda','lda','crlda','alda','bcnn','cnn', 'crcnn2','acnn03','acnn30','acewc30','acewc15', 'adcnn30', 'vcnn', 'bvcnn', 'avcnn', 'avcnn15', 'acnnl03','crvcnn','acewclm','crcnn','acewc00','xtra2']
ft = 'tdar'
iter = 5
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')


for it in range(iter):
    # first iteration, includes LDA; others exclude LDA
    if it == 0:
        mod_all = ['blda','alda','bcnn','lda','cnn','acnn03','avcnn','crcnn']
        # mod_all = ['avcnn']
    else:
        mod_all = ['bcnn','cnn','acnn03','avcnn','crcnn']

    for sub in range(7):
        print(subs[sub])
        sub_path = path + subs[sub] + '/DATA/MAT/'
        all_files = os.listdir(sub_path)
        if 'skip' in all_files:
            all_files = np.delete(all_files,all_files.index('skip'))

        # load or initialize cnn weights
        if load_mod:
            with open(subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
                all_acc, all_recal, all_val, all_prev, all_train, all_times, _, _, c_weights, v_weights, cl_wc, scaler_0, emg_scale = pickle.load(f)
        else:
            c_weights = None
            v_weights = None
            v_wc = None
            cl_wc = None
            scaler_0 = None
            # all_recal = np.empty((len(mod_tot),len(mod_tot)))
            # all_recal[:] = np.nan
            all_recal = np.zeros((len(all_files),len(mod_tot)))
            all_acc = np.zeros((len(all_files),len(mod_tot)))
            all_val = np.zeros((len(all_files),len(mod_tot)))
            all_prev = np.zeros((len(all_files),len(mod_tot)))
            all_train = np.zeros((len(all_files),len(mod_tot)))
            all_times = np.zeros((len(all_files),len(mod_tot)))

        mod_i = 0
        for mod in mod_all:
            acc = np.zeros((len(all_files),2))
            acc_val = np.zeros((len(all_files),2))
            acc_prev = np.zeros((len(all_files),2))
            acc_train = np.zeros((len(all_files),2))
            mod_recal = np.zeros((len(all_files),))

            if 'lda' in mod:
                acc_i = 0
            else:
                acc_i = 1

            cnn = None
            ewc = None
            clda = None

            ep = 30
            recal = 0
            skip_recal = 0
            skip = False

            # Loop through files
            for i in range(0,len(all_files)-1):
                # Check if need to recalibrate
                if i > 0:
                    if acc[i,acc_i] < 75:
                        skip = False
                        recal += 1
                        print('recal: ' + str(recal) + ' ' + all_files[i])
                        acc[i,acc_i] *= -1
                        mod_recal[i] = 1
                    else:
                        skip = True
                    
                    if 'b' in mod:
                        skip = True
                        
                if not skip:
                    # load training file
                    train_file = all_files[i]
                    train_data, train_params = prd.load_caps_train(sub_path + train_file + '/traindata.mat')

                    # if first train, use two train files
                    if i == 0:
                        train_data, train_params, th = prd.threshold(train_data, train_params)
                    else:
                        train_data, train_params, _ = prd.threshold(train_data, train_params,th)
                    
                    tr_i = np.zeros((train_params.shape[0],))
                    te_i = np.zeros((train_params.shape[0],))
                    for cls in np.unique(train_params[:,-1]):
                        dof = np.array(np.where(train_params[:,-1] == cls))
                        tr_i[dof[0,:dof.shape[1]//2]] = 1
                        te_i[dof[0,dof.shape[1]//2:]] = 1

                    train_temp = train_data[tr_i.astype(bool),...]
                    params_temp = train_params[tr_i.astype(bool),...]
                    val_data = train_data[te_i.astype(bool),...]
                    val_params = train_params[te_i.astype(bool),...]

                    train_data, train_params = train_temp, params_temp

                    del train_temp, params_temp, tr_i, te_i

                    if (i == 0 and mod[0] == 'a') or (i == 0 and mod[:2] == 'cr') or (mod[0] != 'a' and mod[:2] != 'cr'):
                        if i > 0:
                            prev_ndof = n_dof
                        train_dof = np.unique(train_params[:,-1])
                        key = np.empty(train_dof.shape)
                        for key_i in range(len(train_dof)):
                            key[key_i] = cp.deepcopy(train_params[np.argmax(train_params[:,2] == train_dof[key_i]),0])
                        n_dof = int(np.max(key))

                    train_data, train_params = lp.check_labels(train_data,train_params,train_dof,key)
                    val_data, val_params = lp.check_labels(val_data,val_params,train_dof,key)

                    for cl in train_dof:
                        train_params[train_params[:,-1] == cl,0] = key[train_dof==cl]
                        val_params[val_params[:,-1] == cl,0] = key[train_dof==cl]

                    print('train dof:' ,end=' ')
                    print(train_dof)

                    if (mod[0] == 'a' and i > 0) or ('cr' in mod and i > 0) or (mod == 'vcnn' and i > 0):
                        _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, _, _, _, _, _ = prd.prep_train_caps(train_data, train_params, emg_scale=emg_scale, scaler=scaler, num_classes=n_dof, prop_b=False, batch_size=bat, ft=ft, noise=True, split=False)
                    else:
                        _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, emg_scale, scaler, _, _, _ = prd.prep_train_caps(train_data, train_params, prop_b=False, batch_size=bat, ft=ft, noise=True, split=False,num_classes=n_dof)
                        if ((i == 0) and (c_weights is not None)) or ((i == 0) and (v_weights is not None)):
                            scaler = cp.deepcopy(scaler_0)

                    _, _, _, _, x_val_cnn, y_val, x_val_lda, y_val_lda, _, _, _, _, _ = prd.prep_train_caps(val_data, val_params, emg_scale=emg_scale,scaler=scaler, prop_b=False, batch_size=bat, ft=ft, num_classes=n_dof, noise=False, split=False)

                    if 'cr' in mod:
                        # combine old and new training data
                        if i > 0:
                            x_clean_cnn = np.vstack((clean_data_0,x_clean_cnn))
                            y_clean = np.vstack((clean_params_0,y_clean))
                            x_train_cnn = np.vstack((x_clean_cnn,x_train_cnn))
                            y_train = np.vstack((y_clean,y_train))

                    del train_data, train_params, val_data, val_params

                    if 'lda' not in mod:
                        cnnlda = 'l' in mod
                        if 'vcnn' in mod:
                            if i == 0:
                                # calculate normalization scale
                                adjust = []

                                old_y = cp.deepcopy(y_clean)

                                if v_weights is None:
                                    cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_clean_cnn,y_train=y_clean, mod=['vcnn'], n_dof=n_dof, ep=30, dec=True, trainable=True)
                                    # save current autoencoder weights
                                    b_enc_w = cp.deepcopy(cnn.enc.get_weights())
                                    dec_w = cp.deepcopy(cnn.dec.get_weights())
                                    v_weights = cp.deepcopy(cnn.dec.get_weights())
                                else:
                                    cnn = dl.VCNN(n_class = n_dof)
                                    cnn(x_train_cnn[:1,...])
                                    cnn.add_dec(x_train_cnn[:1,...])
                                    cnn(x_train_cnn[:2,...],np.ones((2,)),dec=True) 
                                    cnn.dec.set_weights(v_weights)

                                if c_weights is None:
                                    cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=[cnn], n_dof=n_dof, ep=30, dec=False, trainable=True)
                                    c_weights = [cnn.enc.get_weights(),cnn.clf.get_weights()]
                                else:
                                    print('setting CNN weights')
                                    cnn.enc.set_weights(c_weights[0])
                                    cnn.clf.set_weights(c_weights[1])

                                if scaler_0 is None:
                                    scaler_0 = cp.deepcopy(scaler)
                                else:
                                    scaler = cp.deepcopy(scaler_0)

                                mu_class, std_class, N = prd.update_mean(x_clean_cnn,y_clean)

                            else:
                                prev_w = cnn.get_weights()
                                prev_mu = [mu_class, std_class, N]
                                
                                # generate old training data, same size as clean data

                                x_out = cp.deepcopy(x_clean_cnn)
                                for cl in range((y_clean).shape[1]):
                                    x_ind = y_clean[:,cl]==1
                                    x_out[x_ind,...] = np.random.normal(mu_class[cl], std_class[cl],x_clean_cnn[x_ind,...].shape)

                                x_train_aug = np.vstack((x_out,x_train_cnn))
                                y_train_aug = np.vstack((y_clean,y_train))

                                x_clean_aug = np.vstack((x_out,x_clean_cnn))
                                y_clean_aug = np.vstack((y_clean,y_clean))

                                old_y = cp.deepcopy(y_clean_aug)
                                mu_class, std_class, N = prd.update_mean(x_clean_cnn,y_clean,N,mu_class,std_class)


                                if 'avcnn' in mod: # update whole CNN and lda weights
                                    cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_aug,y_train=y_train_aug, mod=[cnn], n_dof=n_dof, ep=5, dec=False, lr=0.00001, trainable=False)
                                else:
                                    cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_aug,y_train=y_train_aug, mod=[cnn], n_dof=n_dof, ep=30, dec=False, trainable=True)

                                    w, c, _, _, _, _, _ = dlda.train_lda(x_train_aug.reshape((x_train_aug.shape[0],-1)), np.argmax(y_train_aug,axis=1)[...,np.newaxis])                            
                        else:
                            if i == 0:
                                if c_weights is None:
                                    cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['cnn'], n_dof=n_dof, ep=ep, cnnlda=cnnlda)
                                    c_weights = cp.deepcopy([cnn.enc.get_weights(),cnn.clf.get_weights()])
                                    scaler_0 = cp.deepcopy(scaler)    
                                else:
                                    print('setting CNN weights')
                                    cnn = dl.CNN(n_class=n_dof)
                                    cnn(x_train_cnn[:1,...])
                                    cnn.enc.set_weights(c_weights[0])
                                    cnn.clf.set_weights(c_weights[1])
                                    if cnnlda:
                                        print('setting LDA weights')
                                        w_c = cp.deepcopy(cl_wc[0].astype('float32'))
                                        c_c = cp.deepcopy(cl_wc[1].astype('float32'))
                                if 'ewc' in mod:
                                    cnn = dl.EWC(n_class=n_dof)
                                    cnn(x_train_cnn[:1,...])
                                    cnn.enc.set_weights(c_weights[0])
                                    cnn.clf.set_weights(c_weights[1])
                                if 'ad' in mod:
                                    cnn = dl.CNN(n_class=n_dof,adapt=True)
                                    cnn(x_train_cnn[:1,...])
                                    cnn.enc.set_weights(c_weights[0])
                                    cnn.clf.set_weights(c_weights[1])

                                if 'l' in mod:
                                    clda = [w_c, c_c]
                                
                                if scaler_0 is None:
                                    scaler_0 = cp.deepcopy(scaler)
                                else:
                                    scaler = cp.deepcopy(scaler_0)

                            else:
                                prev_w = cnn.get_weights()
                                if mod =='acnnlm': # update CNN encoder using lda for loss
                                    ep = 5
                                    cnn, all_times[i,mod_tot.index(mod)], _, _ = lp.train_models(traincnn=x_train_cnn,y_train=y_train, n_dof=n_dof, ep=ep, mod=[[cnn,w_c,c_c]],cnnlda=cnnlda)
                                elif 'adcnn' in mod: # adapt first layer only
                                    # cnn.base.trainable=False
                                    cnn.clf.trainable=False
                                    ep = int(mod[-2:])
                                    cnn, all_times[i,mod_tot.index(mod)], _, _ = lp.train_models(traincnn=x_train_cnn,y_train=y_train, n_dof=n_dof, ep=ep, mod=[cnn], adapt=True, cnnlda=cnnlda, lr=0.00001)
                                elif 'acnn' in mod: # update whole CNN and lda weights
                                    ep = int(mod[-2:])
                                    ep =5
                                    cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, n_dof=n_dof, ep=ep, mod=[cnn], cnnlda=cnnlda, lr=0.00001)
                                elif mod == 'afcnnl': # update lda only 
                                    w_c, c_c = lp.train_models(x_train_lda=cnn.enc(x_train_cnn).numpy(), y_train_lda=np.argmax(y_train,axis=1)[...,np.newaxis], mod=['lda'])
                                elif 'cnn' in mod: # recalibrate cnnlda
                                    cnn, all_times[i,mod_tot.index(mod)], w_c, c_c = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['cnn'], n_dof=n_dof, ep=ep, cnnlda=cnnlda)
                                elif mod == 'acewclm':
                                    _, _, all_times[i,mod_tot.index(mod)] = lp.train_task(cnn, ep, 1, x_train_cnn, y_train, [x_val_cnn], [y_val], lams=[int(mod[-2:])], bat=bat, clda=[w_c,c_c], cnnlda=cnnlda)
                                elif 'acewc' in mod:
                                    w_c, c_c, all_times[i,mod_tot.index(mod)] = lp.train_task(cnn, 15, 1, x_train_cnn, y_train, [prev_x, x_val_cnn],[prev_y, y_val], lams=[int(mod[-2:])], bat=bat, cnnlda=cnnlda)
                                
                                if 'l' in mod:
                                    clda = [w_c, c_c]
                        
                        test_mod = dl.get_test(cnn, test_accuracy)
                        if i > 0:
                            acc_prev[i,:] = lp.test_models(prev_x, prev_y, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)
                        acc_val[i,:] = lp.test_models(x_val_cnn, y_val, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)
                        acc_train[i,:] = lp.test_models(x_clean_cnn, y_clean, None, None, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)

                        if acc_val[i,acc_i] < 75:
                            if 'vcnn' in mod:
                                mu_class, std_class, N = prev_mu
                                cnn = dl.VCNN(n_class = prev_ndof)
                            elif 'cnn' in mod:
                                cnn = dl.CNN(n_class = prev_ndof)
                            cnn(x_train_cnn[:1,...])
                            cnn.set_weights(prev_w)
                            del test_mod
                            test_mod = dl.get_test(cnn, test_accuracy)
                            mod_recal[i] = -1
                            print('bad recal')
                        elif 'cr' in mod:
                            clean_data_0 = cp.deepcopy(x_clean_cnn)
                            clean_params_0 = cp.deepcopy(y_clean)
                        if 'ewc' in mod: 
                            cnn.compute_fisher(x_train_cnn, y_train, num_samples=200, plot_diffs=False) 
                            cnn.star()
                    else:
                        if i == 0:
                            N = np.zeros((len(np.unique(y_train_lda)),))
                            cov_class = np.zeros([x_train_lda.shape[1],x_train_lda.shape[1]])
                            mu_class = np.zeros([len(np.unique(y_train_lda)),x_train_lda.shape[1]])
                        prev_lda = [mu_class,cov_class,N]
                        start_time = time.time()
                        if mod[0] != 'a' or (i == 0 and mod[0] == 'a'):
                            w, c, mu_class, _, _, N, cov_class = dlda.train_lda(x_train_lda, y_train_lda)
                        else:
                            w, c, mu_class, cov_class, N = dlda.update_lda(x_train_lda, y_train_lda, N, mu_class, cov_class)
                        all_times[i,mod_tot.index(mod)] = time.time() - start_time

                        acc_val[i,:] = lp.test_models(None, None, x_val_lda, y_val_lda, lda=[w,c])
                        acc_train[i,:] = lp.test_models(None, None, x_train_lda, y_train_lda, lda=[w,c])
                        if i > 0:
                            acc_prev[i,:] = lp.test_models(None, None, prev_x_lda, prev_y_lda, lda=[w,c])
                        if acc_val[i,acc_i] < 75:
                            mod_recal[i] = -1
                            mu_class, cov_class, N = prev_lda
                            print('bad recal')
                        del x_train_lda, y_train_lda
                    
                    if mod_recal[i] != -1:
                        prev_x = cp.deepcopy(x_val_cnn)
                        prev_y = cp.deepcopy(y_val)
                        prev_x_lda = cp.deepcopy(x_val_lda)
                        prev_y_lda = cp.deepcopy(y_val_lda)
                    
                    del x_train_cnn, y_train, x_val_cnn, y_val, x_val_lda, y_val_lda, x_clean_cnn, y_clean
                
                # load data
                test_file = all_files[i+1]
                test_data, test_params = prd.load_caps_train(sub_path + test_file + '/traindata.mat')
                
                # check class labels
                test_data, test_params, _ = prd.threshold(test_data, test_params, th)
                test_data, test_params = lp.check_labels(test_data,test_params,train_dof,key)

                for cl in train_dof:
                    test_params[test_params[:,-1] == cl,0] = key[train_dof==cl]

                print('test dof:',end=' ')
                print(np.unique(test_params[:,-1]))
                # test 
                y_test, _, x_test_cnn, x_test_lda, y_test_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=n_dof, ft=ft, split=False)
                
                # test 
                if 'lda' in mod:
                    acc[i+1,:] = lp.test_models(None, None,  x_test_lda, y_test_lda, lda=[w,c])
                else:
                    acc[i+1,:] = lp.test_models(x_test_cnn, y_test, x_test_lda, y_test_lda, cnn=cnn, clda=clda, test_mod=test_mod, test_accuracy=test_accuracy)

                print ('Set: ' + train_file + ', Test: ' + test_file + ',', f'Accuracy: {acc[i+1,acc_i]:.2f}', f', Val: {acc_val[i,acc_i]:.2f}', f', Prev: {acc_prev[i,acc_i]:.2f}', f', Train: {acc_train[i,acc_i]:.2f}')
                del y_test, x_test_cnn, x_test_lda, y_test_lda, test_data, test_params

            all_acc[:,mod_tot.index(mod)] = acc[:,acc_i]
            all_val[:,mod_tot.index(mod)] = acc_val[:,acc_i]
            all_prev[:,mod_tot.index(mod)] = acc_prev[:,acc_i]
            all_train[:,mod_tot.index(mod)] = acc_train[:,acc_i]
            all_recal[:,mod_tot.index(mod)] = mod_recal

            print(mod + ' ' + str(recal))
            mod_i += 1

        with open('0326 half init train/' + subs[sub] + '_' + str(it) + '_r_accs.p','wb') as f:
            pickle.dump([all_acc, all_recal, all_val, all_prev, all_train, all_times, mod_all, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale],f)

In [ ]:
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
bat = 32
ft = 'tdar'
iter = 10
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
plot_mod = ['lda','alda','cnn','acnn03','avcnn','crcnn']
plot_mod = ['lda','avcnn']
for plot_i in range(1):
    for sub in range(1):    
        it_acc = []
        it_recal = []
        it_val = []
        it_prev = []
        it_train = []
        it_times = []
        for it in range(iter):
            # load or initialize cnn weights
            if plot_i == 1:
                with open('0323 full run pre and post/' + subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
                    all_acc, all_recal, all_val, all_prev, all_train, all_times, _, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale = pickle.load(f)
            else:
                with open('0325 big run/' + subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
                    all_acc, all_recal, all_val, all_prev, all_train, all_times, _, mod_tot, c_weights, v_weights, cl_wc, scaler_0, emg_scale = pickle.load(f)
            
            all_acc[all_acc==0] = np.nan
            # all_recal[all_recal==0] = np.nan
            all_val[all_val==0] = np.nan
            all_prev[all_prev==0] = np.nan
            all_train[all_train==0] = np.nan
            all_times[all_times==0] = np.nan

            it_acc.append(all_acc)
            it_recal.append(all_recal)
            it_val.append(all_val)
            it_prev.append(all_prev)
            it_train.append(all_train)
            it_times.append(all_times)

            it_acc[it][:,:2] = it_acc[0][:,:2]
            it_recal[it][:2] = it_recal[0][:2]
            it_val[it][:,:2] = it_val[0][:,:2]
            it_prev[it][:,:2] = it_prev[0][:,:2]
            it_train[it][:,:2] = it_train[0][:,:2]
            it_times[it][:,:2] = it_times[0][:,:2]


        it_acc2 = cp.deepcopy(it_acc)
        for i in range(len(it_acc2)):
            x = it_val[i]< 0
            # print(x.type)
            # print(it_acc2[i].shape)
            # print(ave_val.shape)
            it_acc2[i][it_acc[i]< 0]= it_val[i][it_acc[i]< 0]

        ave_acc2 = np.nanmean(np.abs(np.array(it_acc2)),axis=0)
        ave_acc = np.nanmean(np.abs(np.array(it_acc)),axis=0)
        ave_recal = np.nanmean(np.abs(np.array(it_recal)),axis=0)
        ave_val = np.nanmean(np.abs(np.array(it_val)),axis=0)
        ave_prev = np.nanmean(np.abs(np.array(it_prev)),axis=0)
        ave_train = np.nanmean(np.abs(np.array(it_train)),axis=0)
        ave_times = np.nanmean(np.abs(np.array(it_times)),axis=0)

        std_acc2 = np.nanstd(np.abs(np.array(it_acc2)),axis=0)/np.sum(~np.isnan(np.array(it_acc2)),axis=0)
        std_acc = np.nanstd(np.abs(np.array(it_acc)),axis=0)/np.sum(~np.isnan(np.array(it_acc)),axis=0)
        std_recal = np.nanstd(np.abs(np.array(it_recal)),axis=0)/np.sum(~np.isnan(np.array(it_recal)),axis=0)
        std_val = np.nanstd(np.abs(np.array(it_val)),axis=0)/np.sum(~np.isnan(np.array(it_val)),axis=0)
        std_prev = np.nanstd(np.abs(np.array(it_prev)),axis=0)/np.sum(~np.isnan(np.array(it_prev)),axis=0)
        std_train = np.nanstd(np.abs(np.array(it_train)),axis=0)/np.sum(~np.isnan(np.array(it_train)),axis=0)
        std_times = np.nanstd(np.abs(np.array(it_times)),axis=0)/np.sum(~np.isnan(np.array(it_times)),axis=0)

        fig, ax = plt.subplots(1,2,figsize=(20,5))
        for mod in plot_mod:
            plot_ind = mod_tot.index(mod)
            # ax.plot(ave_acc[2:,plot_ind],'.-',label= mod + ' = '+ str(ave_recal[plot_ind,0]) + ' +/- ' + str(std_recal[plot_ind,0]))
            x = ~np.isnan(ave_val[:,plot_ind])
            # ave_acc2 = cp.deepcopy(ave_acc)
            # ave_acc[x,plot_ind] = ave_val[x,plot_ind]

            ax[0].plot(ave_acc[:,plot_ind],'.-',ms=8,label= mod + ' = '+ str(ave_recal[plot_ind,0]) + ' +/- ' + str(std_recal[plot_ind,0]))
            ax[0].plot(np.squeeze(np.where(x)),ave_acc[x,plot_ind],'kx',ms=12)
            ax[1].plot(ave_acc2[:,plot_ind],'.-',ms=8,label= mod + ' = '+ str(ave_recal[plot_ind,0]) + ' +/- ' + str(std_recal[plot_ind,0]))
            # ax.plot(np.squeeze(np.where(x)), ave_val[~np.isnan(ave_val[:,plot_ind]),plot_ind],'.-',ms=8,label= mod + ' = '+ str(ave_recal[plot_ind,0]) + ' +/- ' + str(std_recal[plot_ind,0]))
            # plt.fill_between(np.arange(ave_acc[2:,plot_ind].shape[0]),ave_acc[2:,plot_ind]-std_acc[2:,plot_ind],ave_acc[2:,plot_ind]+std_acc[2:,plot_ind],alpha=.3)
        ax[0].legend()
        for i in range(2):
            ax[i].axhline(75, ls='--', color='grey')
            ax[i].set_ylim([0,100])
        # ax[1].axhline(75, ls='--', color='grey')

In [ ]:
np.sum(ave_val < 75,axis=0)

In [ ]:
mod_tot

## Test generation

In [ ]:
# Count recalibrations
path = 'C:/Users/yteh/Documents/work/necal/home data/'
subs = os.listdir(path)
if 'skip' in subs:
    subs = np.delete(subs,subs.index('skip'))
bat = 128
load_mod = False
mod_tot = ['blda','lda','crlda','alda','bcnn','cnn', 'crcnn','acnn03','acnn30','acewc00','acewc30', 'adcnn30', 'vcnn', 'bvcnn', 'avcnn03', 'avcnn15', 'acnnl03','crvcnn','acewclm','xtra','xtra1','xtra2']
ft = 'feat'
iter = 1

for sub in range(4,5):
    print(subs[sub])
    sub_path = path + subs[sub] + '/DATA/MAT/'
    all_files = os.listdir(sub_path)
    if 'skip' in all_files:
        all_files = np.delete(all_files,all_files.index('skip'))

    # first iteration, includes LDA; others exclude LDA
    mod_all = ['vcnn']

    # load or initialize cnn weights
    if load_mod:
        with open(subs[sub] + '_' + str(0) + '_r_accs.p','rb') as f:
            all_acc, all_recal, all_val, all_prev, all_train, all_times, _, _, c_weights, cl_wc, scaler_0, emg_scale = pickle.load(f)
        all_times = np.zeros((len(all_files),len(mod_tot)))
    else:
        c_weights = None
        v_weights = None
        v_wc = None
        cl_wc = None
        all_recal = np.empty((len(mod_tot),1))
        all_recal[:] = np.nan
        all_acc = np.zeros((len(all_files),len(mod_tot)))
        all_val = np.zeros((len(all_files),len(mod_tot)))
        all_prev = np.zeros((len(all_files),len(mod_tot)))
        all_train = np.zeros((len(all_files),len(mod_tot)))
        all_times = np.zeros((len(all_files),len(mod_tot)))

    mod_i = 0
    for mod in mod_all:
        acc = np.zeros((len(all_files),5))
        acc_val = np.zeros((len(all_files),5))
        acc_prev = np.zeros((len(all_files),5))
        acc_train = np.zeros((len(all_files),5))

        if 'cnn' in mod:
            acc_i = 2
        elif 'cewc' in mod:
            acc_i = 4
        elif 'lda' in mod:
            acc_i = 0

        cnn = None
        ewc = None

        ep = 50
        recal = 0
        skip = False

        # Loop through files
        for i in range(1,2):#len(all_files)-1):
            # load training file
            train_file = all_files[i]
            train_data, train_params = prd.load_caps_train(sub_path + train_file + '/traindata.mat')

            train_data, train_params, th = prd.threshold(train_data, train_params)
            val_data = train_data
            val_params = train_params

            train_dof = np.unique(train_params[:,-1])
            key = np.empty(train_dof.shape)
            for key_i in range(len(train_dof)):
                key[key_i] = cp.deepcopy(train_params[np.argmax(train_params[:,2] == train_dof[key_i]),0])
            n_dof = int(np.max(key))
            
            train_data, train_params = lp.check_labels(train_data,train_params,train_dof,key)
            val_data, val_params = lp.check_labels(val_data,val_params,train_dof,key)

            _, x_clean_cnn, y_clean, _, x_train_cnn, y_train, x_train_lda, y_train_lda, emg_scale, scaler, _, _, _ = prd.prep_train_caps(train_data, train_params, prop_b=False, batch_size=bat, ft=ft, noise=True, split=False,num_classes=n_dof)

            _, _, _, _, x_val_cnn, y_val, x_val_lda, y_val_lda, _, _, _, _, _ = prd.prep_train_caps(val_data, val_params, emg_scale=emg_scale,scaler=scaler, prop_b=False, batch_size=bat, ft=ft, num_classes=n_dof, noise=False, split=False)

            del train_data, train_params, val_data, val_params

            cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['vcnn'], n_dof=n_dof, ep=30, dec=True, print_b=True)
            cnn, all_times[i,mod_tot.index(mod)] = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=[cnn], n_dof=n_dof, ep=30, dec=False,print_b=True)

In [ ]:
x_out = cp.deepcopy(x_clean_cnn)
for cl in range(y_clean.shape[1]):
    x_out[y_clean[:,cl]==1,...] = np.random.normal(np.mean(x_clean_cnn[y_clean[:,cl]==1,...],axis=0), np.std(x_clean_cnn[y_clean[:,cl]==1,...],axis=0),x_clean_cnn[y_clean[:,cl]==1,...].shape)


In [ ]:
x_out = cp.deepcopy(x_train_cnn)
for cl in range(y_train.shape[1]):
    x_out[y_train[:,cl]==1,...] = np.random.normal(np.mean(x_train_cnn[y_train[:,cl]==1,...],axis=0), np.std(x_train_cnn[y_train[:,cl]==1,...],axis=0),x_train_cnn[y_train[:,cl]==1,...].shape)


In [ ]:
samp1 = np.ones((1*y_clean.shape[0],8))
x_out,_,_ = cnn.dec(samp1,np.tile(np.argmax(y_clean,axis=1),[1]).astype('float32'),samp=True)
# _,x_out,_,_ = cnn(x_clean_cnn,np.tile(np.argmax(y_clean,axis=1),[1]).astype('float32'),dec=True)
x_out = x_out.numpy()
# for i in range(y_clean.shape[1]):
#     adjust1 = np.std(x_clean_cnn[np.argmax(y_clean,axis=1)==i,...],axis=0)
#     rescale = np.mean(adjust1)/np.mean(np.std(x_out[np.argmax(y_clean,axis=1)==i,...],axis=0))
#     gmean = np.mean(x_out[np.argmax(y_clean,axis=1)==i,...],axis=0)
#     x_out[np.argmax(y_clean,axis=1)==i,...] = (x_out[np.argmax(y_clean,axis=1)==i,...] - gmean)*rescale + gmean
# x_out = np.maximum(np.minimum(x_out,1),0)
for cl in range(y_clean.shape[1]):
    ind = np.tile(np.argmax(y_clean,axis=1)==cl,[1])
    ind2 = np.argmax(y_clean,axis=1)==cl
    x_temp = x_out[ind,...].reshape((np.sum(ind),-1))
    x_true = x_clean_cnn[ind2,...].reshape((np.sum(ind2),-1))
    # for i in range()
    plt.figure()
    # for i in range(x_true.shape[0]):
    #     plt.plot(x_true[i,...],'k-')
        
    for i in range(x_temp.shape[0]):
        plt.plot(x_temp[i,...],'b-')

In [ ]:
x_lda = x_clean_cnn.reshape(x_clean_cnn.shape[0],-1)
y_lda = np.argmax(y_clean,axis=1)[...,np.newaxis]
y_train_lda = np.argmax(y_clean,axis=1)[...,np.newaxis]
w,c, _, _, _, _, _ = dlda.train_lda(x_lda,y_lda)
y_out = dlda.predict(x_lda, w, c)
print(dlda.eval_lda(w, c, x_lda, y_lda))
x_out_lda = x_out.reshape(x_out.shape[0],-1)
print(dlda.eval_lda(w,c, x_out_lda,np.tile(y_train_lda,[1,1])))
w,c, _, _, _, _, _ = dlda.train_lda(x_out_lda,np.tile(y_train_lda,[1,1]))
print(dlda.eval_lda(w, c, x_out_lda,y_train_lda))
print(dlda.eval_lda(w, c, x_lda, np.argmax(y_clean,axis=1)[...,np.newaxis]))

In [ ]:
cnn1, _ = lp.train_models(traincnn=x_train_cnn,y_train=y_train, mod=['vcnn'], n_dof=n_dof, ep=15, dec=False,print_b=True)
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
test_mod = dl.get_test(cnn1,test_accuracy)
print(lp.test_models(x_out, y_clean, None, None, cnn=cnn1, test_mod=test_mod, test_accuracy=test_accuracy))

cnn2, _ = lp.train_models(traincnn=x_out,y_train=y_clean, mod=['vcnn'], n_dof=n_dof, ep=15, dec=False,print_b=True)
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
test_mod = dl.get_test(cnn2,test_accuracy)
print(lp.test_models(x_clean_cnn, y_clean, None, None, cnn=cnn2, test_mod=test_mod, test_accuracy=test_accuracy))

In [ ]:
cv_iter = 1
fig,ax = plt.subplots(1,5,figsize=(30,4))
for sub in range(2,3):#,5):
    with open(subs[sub] + '_0_r_accs.p','rb') as f:
        acc_all, recal_all, cur_all, prev_all, val_all, mod_all, mod_tot, c_weights, cl_wc, scaler_0, emg_scale= pickle.load(f)

    # mod_all = ['ld','bld','bcnnl','cnnl','acnnl','acnnl3','acnnl30','acewcl']
    colors =  cm.get_cmap('tab20c')
    c = np.empty((20,4))
    for i in range(20):
        c[i,:] = colors(i*1/20)


    nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[9,:],c[1,:],c[10,:],c[2,:]))
    nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:]))
    nn_c[0,-1] = 1
    all_m = ['ko-','o-','o-','s-','s-','v-','v-']
    pt_m = ['ko-','o-','o-','o-','s','s','s','s','D']
    nn_c = np.vstack((np.array([0,0,0,1]), c[0,:],c[1,:],c[2,:],c[3,:],c[4,:],c[5,:],c[6,:],c[0,:],c[8,:],c[0,:]))
    # nn_c[0,-1] = 1

    labels = ['lda','mlp','cnn','a-mlp','a-cnn','ewc-mlp','ewc-cnn','c-mlp','c-cnn','c-ld']
    labels = ['r-lda','lda','cnn','r-cnn','a-cnn-5','a-cnn-3','a-cnn-30','ewc-cnn']
    # labels = mod_tot

    ax_ind = sub
    it = 0
    for v in [1,2]: 
        i = mod_tot.index(mod_all[v])
        acc_temp = acc_all[1:-1,i]
        if not np.isnan(acc_temp).all():
            x = np.arange(len(acc_temp))
            recal_i = (acc_temp < 0)
            ax[ax_ind].plot(np.abs(acc_temp),'-',color=nn_c[it,:])
            ax[ax_ind].plot(x[~recal_i],np.abs(acc_temp[~recal_i]),pt_m[it],label=labels[v],color=nn_c[it,:])
            ax[ax_ind].plot(x[recal_i],np.abs(acc_temp[recal_i]),'x',ms=10,color=nn_c[it,:])
            it+=1

    for i in range(5):
        ax[i].axhline(70, ls='--', color='grey')
        
        ax[i].set_ylim([0,100])
        ax[i].set_title('TR' + str(i+1))
    ax[0].legend()
    ax[2].set_xlabel('Calibration Set')
    ax[0].set_ylabel('Accuracy (%)')
    plt.rc('font', size=15) 

In [ ]:
cv_iter = 1
for sub in range(2,3):#,5):
    fig,ax = plt.subplots(1,4,figsize=(20,4))
    for it in range(0,cv_iter):
        with open(subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
            # acc_all, recal_all = pickle.load(f)
            acc_all, recal_all, cur_all, prev_all, val_all,mod_all, mod_tot, c_weights, cl_wc, scaler_0, emg_scale= pickle.load(f)

        # mod_all = ['ld','bld','bcnnl','cnnl','acnnl','acnnl3','acnnl30','acewcl']
        colors =  cm.get_cmap('tab20c')
        c = np.empty((20,4))
        for i in range(20):
            c[i,:] = colors(i*1/20)


        nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[9,:],c[1,:],c[10,:],c[2,:]))
        nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:]))
        nn_c[0,-1] = 1
        all_m = ['ko-','o-','o-','s-','s-','v-','v-']
        pt_m = ['ko','*','*','o','s','s','s','s','D']
        pt_m = ['ko','*','*','o','s','s','s','s','D']
        nn_c = np.vstack((np.array([0,0,0,1]),np.array([0,0,0,1]), c[0,:],c[1,:],c[4,:],c[5,:],c[6,:],c[8,:],c[6,:],c[0,:],c[8,:],c[0,:]))

        labels = ['lda','mlp','cnn','a-mlp','a-cnn','ewc-mlp','ewc-cnn','c-mlp','c-cnn','c-ld']
        labels = ['r-lda','lda','cnn','r-cnn','a-cnn-5','a-cnn-3','a-cnn-30','ewc-cnn']
        # labels = mod_tot

        ax_ind = [1,0,0,1,2,2,2,2,2,2,2,2,3,3,3,3]
        it = 0
        for v in range(len(mod_all)):
            i = mod_tot.index(mod_all[v])
            acc_temp = acc_all[1:-1,i]
            if not np.isnan(acc_temp).all():
                x = np.arange(len(acc_temp))
                recal_i = (acc_temp < 0)
                ax[ax_ind[it]].plot(np.abs(acc_temp),'-',color=nn_c[it,:])
                ax[ax_ind[it]].plot(x[~recal_i],np.abs(acc_temp[~recal_i]),pt_m[it],label=labels[it]+': ' + str(int(recal_all[i,0])),color=nn_c[it,:])
                ax[ax_ind[it]].plot(x[recal_i],np.abs(acc_temp[recal_i]),'x',ms=10,color=nn_c[it,:])
                it+=1

    for i in range(4):
        ax[i].axhline(70, ls='--', color='grey')
        ax[i].legend()
        ax[i].set_ylim([0,100])
    ax[0].set_ylabel('Accuracy (%)')

In [ ]:
from matplotlib.ticker import MaxNLocator
cv_iter = 1
for sub in range(0,5):
    fig,ax = plt.subplots(1,2,figsize=(15,5))
    for it in range(0,cv_iter):
        with open(subs[sub] + '_' + str(it) + '_r_accs.p','rb') as f:
            # acc_all, recal_all = pickle.load(f)
            acc_all, recal_all, cur_all, prev_all, mod_all, mod_tot, c_weights, cl_wc, scaler_0, emg_scale= pickle.load(f)

        # mod_all = ['ld','bld','bcnnl','cnnl','acnnl','acnnl3','acnnl30','acewcl']
        colors =  cm.get_cmap('tab20c')
        c = np.empty((20,4))
        for i in range(20):
            c[i,:] = colors(i*1/20)


        nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[9,:],c[1,:],c[10,:],c[2,:]))
        nn_c = np.vstack((np.zeros((1,4)),c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:],c[8,:],c[0,:]))
        nn_c[0,-1] = 1
        all_m = ['ko-','o-','o-','s-','s-','v-','v-']
        pt_m = ['ko','o','*','o','s','D','s','s','D']
        nn_c = np.vstack((np.array([0,0,0,1]),np.array([0,0,0,1]), c[0,:],c[1,:],c[4,:],c[5,:],c[6,:],c[8,:],c[6,:],c[0,:],c[8,:],c[0,:]))

        labels = ['lda','mlp','cnn','a-mlp','a-cnn','ewc-mlp','ewc-cnn','c-mlp','c-cnn','c-ld']
        labels = ['r-lda','lda','cnn','r-cnn','a-cnn-5','a-cnn-3','a-cnn-30','ewc-cnn']
        labels = ['r-lda','lda','cnn','r-cnn','f-cnn-5','f-cnn-3','f-cnn-30','ewc-cnn']
        # labels = mod_tot

        ax_ind = [0,0,1,1,1,1,2,2,2,2,2,2,3,3,3,3]
        it = 0
        for v in [0, 3, 5, 4, 6, 7]: #range(len(mod_all)):
            i = mod_tot.index(mod_all[v])
            acc_temp = acc_all[1:-1,i]
            if not np.isnan(acc_temp).all():
                x = np.arange(len(acc_temp))
                recal_i = (acc_temp < 0)
                ax[ax_ind[it]].plot(np.abs(acc_temp),'-',color=nn_c[v,:])
                ax[ax_ind[it]].plot(x[~recal_i],np.abs(acc_temp[~recal_i]),pt_m[it],label=labels[v]+': ' + str(int(recal_all[i,0])),color=nn_c[v,:])
                ax[ax_ind[it]].plot(x[recal_i],np.abs(acc_temp[recal_i]),'x',ms=10,color=nn_c[v,:])
                ax[ax_ind[it]].xaxis.set_major_locator(MaxNLocator(integer=True))
                it+=1

    for i in range(2):
        ax[i].axhline(70, ls='--', color='grey')
        ax[i].legend()
        ax[i].set_ylim([0,100])
        

    ax[0].set_ylabel('Accuracy (%)')